# QTM 151 - Introduction to Statistical Computing II
## Lecture 13 - Merge Tables in SQL
**Author:** Danilo Freire (danilo.freire@emory.edu, Emory University)

# Hello again! 😊 <br>

# Brief recap 📚

## Recap of last class and today's plan

### Last time we learned how to:

- Connect SQL with Python with `sqlite3` and `pandas`
- Use many SQL commands, such as `CASE WHEN`, window functions, and string functions
- Fill missing data with `COALESCE` and `CASE WHEN`
- Use `pandas` to write and run SQL queries
- Pivot tables in SQLite

### Today we will learn how to:

- See different types of join in SQL
- Use special joins, such as `CROSS JOIN` and `SELF JOIN`
- Merge tables by row with `UNION`, `INTERSECT`, and `EXCEPT`
- Learn how to create `UPSERT` statements in SQLite
- Create `VIEWS` in SQLite
- Solve exercises to practice what we learned
- Let's get started! 🚀

# Basic joins 📊

## Primary and foreign keys

- As with many languages, you can merge two tables in SQL either **by columns or by rows**
- The most common method is the `JOIN` clause
- `JOIN` is used to **combine rows and columns from two or more tables** based on a related column between them
- As you know, there are two types of keys, **primary and foreign keys**
- The **primary key** is a column that **uniquely identifies each row in a table**
- A **foreign key** is a column that **identifies a column in another table**
  - One table can have **multiple foreign keys, and they can be `NULL`**
  - SQLite supports foreign keys, and the Python `sqlite3` module handles them correctly by default

![](figures/common_database_keys_explained-f_mobile.png)

## Load the libraries and connect to the database {#sec:tables}

- Let's load the libraries and connect to the SQLite database. We'll use a file named `lecture13.db`.

In [ ]:
import pandas as pd
import sqlite3
from IPython.display import display # For richer display in Jupyter

# Connect to the SQLite database (this will create the file if it doesn't exist)
connection = sqlite3.connect('lecture13.db')
cursor = connection.cursor()

cursor.execute('DROP TABLE IF EXISTS players;')
cursor.execute('''
CREATE TABLE players (
    player_id INTEGER PRIMARY KEY AUTOINCREMENT, 
    player_name TEXT NOT NULL UNIQUE,
    goals INT NOT NULL, 
    victories INT NOT NULL
);
''')

cursor.execute('DROP TABLE IF EXISTS teams;')
cursor.execute('''
CREATE TABLE teams (
    team_id INTEGER PRIMARY KEY AUTOINCREMENT, 
    team_name TEXT NOT NULL
);
''')
connection.commit() # Commit changes
print("Tables 'players' and 'teams' created (or recreated).")

## Create the tables

- Now let's insert some data into the tables!

In [ ]:
# Insert data into the tables
cursor.execute('''
INSERT INTO players (player_name, goals, victories) VALUES
('Messi', 10, 5),
('Vini Jr', 8, 4),
('Neymar', 6, 3),
('Mbappe', 5, 2),
('Lewandowski', 4, 1),
('Haaland', 5, 3);
''')

cursor.execute('''
INSERT INTO teams (team_name) VALUES
('Inter Miami'),
('Real Madrid'),
('Santos'),
('Real Madrid'),
('Bayern');
''')
connection.commit() # Commit changes
print("Data inserted into 'players' and 'teams'.")

## Visualise the tables

- Let's see our tables using `pandas`. 
- `read_sql` works fine with the `sqlite3` connection object.

In [ ]:
print("Players Table:")
display(pd.read_sql('SELECT * FROM players', connection))

In [ ]:
print("\nTeams Table:")
display(pd.read_sql('SELECT * FROM teams', connection))

## Types of joins

![](figures/joins.webp)

## Inner join

- The `INNER JOIN` returns only the records where there is a match between both tables (intersection) based on the join condition.
- If there's no match for a record in either table, that record will be excluded from the results.
- The matching condition is specified in the `ON` clause (e.g., `ON table1.id = table2.id`).
- Note that Haaland (`player_id` 6) is not in the `teams` table (which only has 5 rows), so he will not appear in the result set.

In [ ]:
print("Inner Join Result:")
display(pd.read_sql('''
SELECT players.player_name, teams.team_name, players.goals, players.victories
FROM players
INNER JOIN teams
ON players.player_id = teams.team_id;
''', connection))

## Left join

- The `LEFT JOIN` returns all records from the left table (first table) and the matched records from the right table (second table).
- The result is `NULL` (`None`) for columns from the right side if there is no match.
- This is perhaps the most common type of join, as it keeps all the data from the table we are usually primarily interested in (the "left" table).
- Note that Haaland is included here because he is in the `players` table (the left table).

In [ ]:
print("Left Join Result:")
display(pd.read_sql('''
SELECT players.player_name, teams.team_name, players.goals
FROM players
LEFT JOIN teams
ON players.player_id = teams.team_id;
''', connection))

## Right join

- The `RIGHT JOIN` returns all records from the right table (second table) and the matched records from the left table (first table).
- The result is `NULL` for columns from the left side if there is no match.
- As you have probably guessed, this is the opposite of the `LEFT JOIN` (and less common).
- In our case, since `teams` has fewer rows than `players` and all `team_id`s match a `player_id`, the `RIGHT JOIN` looks the same as the `INNER JOIN`. 
- *Note: SQLite does not directly support `RIGHT JOIN` or `FULL OUTER JOIN`. These can be emulated using `LEFT JOIN` and `UNION ALL` or by swapping table order for `RIGHT JOIN`.* For simplicity, we'll show the conceptual SQL. If running directly in SQLite, you'd need the workaround.

In [ ]:
print("Right Join Result (conceptual - SQLite would need emulation):")
# SQLite does not directly support RIGHT JOIN. 
# To achieve a similar result, you can swap the tables and use a LEFT JOIN:
display(pd.read_sql('''
SELECT players.player_name, teams.team_name, players.goals
FROM teams -- Swapped to teams as left table
LEFT JOIN players -- Use LEFT JOIN
ON players.player_id = teams.team_id;
''', connection))

## Full outer join

- The `FULL OUTER JOIN` returns all records when there is a match in either the left (first) or right (second) table.
- In my experience, this is the least common type of join.
- Why? Because it returns all the data from both tables, which can be large and potentially less focused. It can also be more computationally intensive.
- The syntax is: `SELECT columns FROM table1 FULL OUTER JOIN table2 ON table1.column = table2.column`
- *Note: SQLite does not directly support `FULL OUTER JOIN`. It needs to be emulated using `LEFT JOIN`, `UNION ALL`, and a `WHERE` clause to exclude duplicates from the right table part of the union.*

In [ ]:
print("Full Outer Join Result (emulated for SQLite):")
full_outer_join_query = '''
SELECT p.player_name, t.team_name, p.goals
FROM players p
LEFT JOIN teams t ON p.player_id = t.team_id
UNION ALL
SELECT p.player_name, t.team_name, p.goals
FROM teams t
LEFT JOIN players p ON p.player_id = t.team_id
WHERE p.player_id IS NULL;
'''
display(pd.read_sql(full_outer_join_query, connection))

## Try it yourself! 🧠 {#sec:exercise01a}

- Let's create two new tables (`products`, `reviews`) and insert some data into them. We use `REAL` for the price in SQLite.

In [ ]:
# Create the tables and insert data
cursor.execute('DROP TABLE IF EXISTS reviews;') 
cursor.execute('DROP TABLE IF EXISTS products;')
cursor.execute('''
CREATE TABLE products (
    product_id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_name TEXT NOT NULL,
    price REAL 
);
''')

# Insert products
cursor.execute('''
INSERT INTO products (product_name, price) VALUES
    ('Coffee Maker', 99.99),
    ('Toaster', 29.99),
    ('Blender', 79.99),
    ('Microwave', 149.99),
    ('Air Fryer', 89.99);
''')

cursor.execute('''
CREATE TABLE reviews (
    review_id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_id INT,
    rating INT CHECK (rating BETWEEN 1 AND 5),
    comment TEXT,
    FOREIGN KEY (product_id) REFERENCES products(product_id)
);
''')

# Insert reviews
cursor.execute('''
INSERT INTO reviews (product_id, rating, comment) VALUES
    (1, 5, 'Great coffee maker!'),
    (1, 4, 'Good but expensive'),
    (2, 3, 'Average toaster'),
    (3, 5, 'Best blender ever');
''')
connection.commit()
print("Tables 'products' and 'reviews' created and populated.")

## Try it yourself! 🧠 {#sec:exercise01b}

- Now try to merge the `products` and `reviews` tables using `INNER JOIN` and `LEFT JOIN`.
- Explain the differences between the two results based on which products appear.

*(Provide the SQL commands and your explanation in the cells below)*

In [ ]:
# Your SQL for INNER JOIN here
# print("INNER JOIN Results (Products with Reviews):")
# display(pd.read_sql(''' ... ''', connection))

In [ ]:
# Your SQL for LEFT JOIN here
# print("\nLEFT JOIN Results (All Products, Reviews if Available):")
# display(pd.read_sql(''' ... ''', connection))

# Special joins 🌟

## Cross join

- `CROSS JOIN` is available in SQL, including SQLite.
- A cross join does not use any comparisons (like `ON`) to match rows.
- Instead, the result is constructed by pairing *every* row from the first table with *every* row from the second table (Cartesian product).
- Useful for generating all possible combinations (e.g., pairing all sizes with all colors).

In [ ]:
# Displaying cross join between players and teams
print("Cross Join between players and teams:")
display(pd.read_sql('''
SELECT players.player_name, teams.team_name
FROM players
CROSS JOIN teams
ORDER BY players.player_id, teams.team_id;
''', connection))

## Cross join

- Here's another example generating T-shirt combinations. SQLite uses `||` for string concatenation, not `CONCAT()`.

In [ ]:
# Drop and recreate tables
cursor.execute('DROP TABLE IF EXISTS colours;')
cursor.execute('DROP TABLE IF EXISTS sizes;')
cursor.execute('CREATE TABLE colours (colour_name TEXT);')
cursor.execute('CREATE TABLE sizes (size_code TEXT);')
cursor.execute("INSERT INTO colours VALUES ('Black'), ('Red');")
cursor.execute("INSERT INTO sizes VALUES ('S'), ('M');")
connection.commit()
print("Created 'colours' and 'sizes' tables and inserted data.")

# Perform cross join and concatenate strings using ||
print("\nT-shirt combinations (Cross Join):")
display(pd.read_sql('''
SELECT
    colours.colour_name,
    sizes.size_code,
    colours.colour_name || ' - ' || sizes.size_code as t_shirt
FROM colours
CROSS JOIN sizes
ORDER BY colours.colour_name, sizes.size_code DESC;
''', connection))

## Self join

- A self join is a regular join, but **the table is joined with itself** (!) 🤯
- It may not be immediately apparent how this could be useful, but it actually has many applications.
- Often, tables describe entities that can have relationships with other entities *of the same type* within that table.
- For instance, if you have a table of `employees`, each row could contain a `manager_id` column that references the `employee_id` of another employee in the same table.
- A self join allows you to connect these related rows, for example, to list each employee alongside their manager's name.

- Since self joins reference the same table twice, **table aliases are required** to distinguish between the two instances of the table.
- You could join the `employees` table like this: `employees AS e JOIN employees AS m ON e.manager_id = m.employee_id`.
- This way, you can clearly specify which instance (`e` for employee, `m` for manager) you are referring to in the `SELECT` list and the `ON` condition.
- The general syntax is: `SELECT columns FROM table1 AS alias1 JOIN table1 AS alias2 ON alias1.column = alias2.column`

## Self join

- Let's see an example with a `family` table where `mother_id` refers back to `person_id`.

In [ ]:
cursor.execute('DROP TABLE IF EXISTS family;')
cursor.execute('''
CREATE TABLE family (
    person_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    mother_id INT 
);
''')

cursor.execute('''
INSERT INTO family (name, mother_id) VALUES
    ('Emma', NULL), -- grandmother (id 1)
    ('Sarah', 1),   -- Emma's daughter (id 2)
    ('Lisa', 1),    -- Emma's daughter (id 3)
    ('Tom', 2),     -- Sarah's son (id 4)
    ('Alice', 2);   -- Sarah's daughter (id 5)
''')
connection.commit()
print("Created 'family' table and inserted data.")

# Self join to find child-mother pairs
print("\nChild-Mother Pairs (Self Join):")
display(pd.read_sql('''
SELECT
    children.name as child,
    mothers.name as mother
FROM family AS children
JOIN family AS mothers ON children.mother_id = mothers.person_id
ORDER BY mothers.name, children.name;
''', connection))

## Self join

- Let's see another example using the `players` table.
- Here we want to compare the `goals` of every player against every other player. We use `p1.player_id < p2.player_id` to avoid duplicate pairs (e.g., Messi vs Vini Jr. and Vini Jr. vs Messi) and comparing a player to themselves.

In [ ]:
print("Comparing Player Goals (Self Join):")
display(pd.read_sql('''
SELECT
    p1.player_name,
    p1.goals,
    p2.player_name as compared_to,
    p2.goals as their_goals,
    p1.goals - p2.goals as difference
FROM players AS p1
JOIN players AS p2
ON p1.player_id < p2.player_id -- Avoid duplicates and self-comparison
ORDER BY difference DESC;
''', connection))

# Merge tables by row 🧩

## Union

- The `UNION` operator combines the result sets of two or more `SELECT` statements vertically (stacking rows).
- It automatically **removes duplicate rows** from the combined results. If you want to keep duplicates, use `UNION ALL`.
- The columns in each `SELECT` statement must be compatible: the same number of columns, and corresponding columns must have compatible data types.
- Let's find all players who have either scored more than 7 goals OR achieved more than 3 victories. `UNION` is suitable here because we are combining two subsets of the *same* entity type (players) based on different criteria, and we want a unique list. 

In [ ]:
# Select players with more than 7 goals OR more than 3 victories
print("UNION: Players with goals > 7 OR victories > 3 (Unique Results)")
display(pd.read_sql('''
SELECT player_name, goals, victories
FROM players
WHERE goals > 7 

UNION -- Combines results and removes duplicates

SELECT player_name, goals, victories
FROM players
WHERE victories > 3

ORDER BY player_name;
''', connection))

## Union all and intersect

- Similar to `UNION`, `UNION ALL` also merges tables by rows (stacks results vertically).
- Unlike `UNION`, `UNION ALL` **retains all duplicate rows**. It simply appends the results. It's generally faster than `UNION` as it doesn't need to check for duplicates.

In [ ]:
# Combine players with > 7 goals and players with > 3 victories
# UNION ALL keeps all rows, including duplicates if a player meets both criteria.
print("UNION ALL: Players with goals > 7 OR victories > 3 (All Results, Duplicates Kept)")
display(pd.read_sql('''
SELECT player_name, goals, victories, 'High Scorer (>7)' AS category
FROM players
WHERE goals > 7

UNION ALL

SELECT player_name, goals, victories, 'Many Victories (>3)' AS category
FROM players
WHERE victories > 3

ORDER BY player_name, category;
''', connection))

## Intersect

- The `INTERSECT` operator returns only the rows that are **common** to the result sets of both `SELECT` statements. It finds the intersection.
- Like `UNION`, it removes duplicates within the final result.
- Let's find players who are both high scorers (more than 7 goals) and have achieved many victories (more than 3).

In [ ]:
# Find players who are BOTH high scorers (goals > 9) AND have many victories (victories > 3)
print("INTERSECT Example: Players with goals > 9 AND victories > 3")
display(pd.read_sql('''
SELECT player_name
FROM players
WHERE goals > 9

INTERSECT

SELECT player_name
FROM players
WHERE victories > 3

ORDER BY player_name;
''', connection))

## Except

- `EXCEPT` returns the rows from the first `SELECT` statement's result set that are **not present** in the second `SELECT` statement's result set. It finds the difference.
- Like `UNION`, it removes duplicates before returning the final result.

In [ ]:
# Example for EXCEPT: Find players who scored more than 5 goals
# but did NOT achieve more than 3 victories.

print("EXCEPT Example: Players with goals > 5 but victories <= 3")
display(pd.read_sql('''
SELECT player_name 
FROM players
WHERE goals > 5

EXCEPT 

SELECT player_name
FROM players
WHERE victories > 3

ORDER BY player_name;
''', connection))

## UPSERT (`INSERT ... ON CONFLICT`)

- SQLite provides "UPSERT" (Update or Insert) operations using the `INSERT ... ON CONFLICT` clause.
- This allows you to attempt an `INSERT`, and if it violates a constraint (like `UNIQUE` or `PRIMARY KEY`), you can specify an alternative action, typically a `DO UPDATE`.
- You can specify different actions for different types of conflicts, such as `IGNORE`, `REPLACE`, or `UPDATE`.
- Let's see a simplified example: If we try to insert a player with an existing `player_name`, we update their `goals` instead.
- You will notice that `excluded` is a special table that refers to the row that would have been inserted if there was no conflict. **SQLite always uses this name**.

In [ ]:
player_data = ('Messi', 2, 1) # player_name, goals_to_add, victories_to_add

sql_upsert = """ 
INSERT INTO players (player_name, goals, victories) VALUES (?, ?, ?) 
ON CONFLICT(player_name) DO UPDATE SET 
goals = goals + excluded.goals, 
victories = victories + excluded.victories;
"""
cursor.execute(sql_upsert, player_data)
connection.commit() # Commit the upsert
print("UPSERT operation for Messi:")
display(pd.read_sql('SELECT * FROM players WHERE player_name = "Messi"', connection))

# Views 🔎

## Views

- A `VIEW` is a virtual table based on the result of a `SELECT` query.
- It does not store data itself but provides a way to simplify complex queries or encapsulate frequently used queries.
- You can create a view using the `CREATE VIEW` statement, followed by the view name and the `SELECT` query.
- In SQLite, you can create a view using the `CREATE VIEW` statement, and you can also drop it using `DROP VIEW`.

In [ ]:
# Drop the view if it exists
cursor.execute('DROP VIEW IF EXISTS player_stats;')

# Create the view
cursor.execute('''
CREATE VIEW player_stats AS
SELECT player_name, SUM(goals) AS total_goals, SUM(victories) AS total_victories
FROM players
GROUP BY player_name;
''')
connection.commit()
print("View 'player_stats' created.")

print("\nContents of player_stats view (LIMIT 4):")
display(pd.read_sql('SELECT * FROM player_stats LIMIT 4', connection))

## Views

- You can also use the view in a join with another table. For example, let's join the `player_stats` view with the `teams` table to see which players are in which teams.
*(Note: The original slide example for `colour_size` view is slightly different from the `player_stats` example. Here we'll use the `player_stats` view for a join example, assuming `player_id` was part of `player_stats` or we join on `player_name` if it's unique and present in both relevant tables. For a more robust join, the view should ideally include the join key.)*

In [ ]:
# To make a meaningful join with teams, player_stats view should ideally contain player_id
# Let's redefine player_stats to include player_id for a better join example
cursor.execute('DROP VIEW IF EXISTS player_stats_with_id;')
cursor.execute('''
CREATE VIEW player_stats_with_id AS
SELECT player_id, player_name, SUM(goals) AS total_goals, SUM(victories) AS total_victories
FROM players
GROUP BY player_id, player_name; -- Group by id and name to keep player_id
''')
connection.commit()
print("View 'player_stats_with_id' created.")

print("\nJoining player_stats_with_id with teams:")
join_view_query = '''
SELECT ps.player_name, t.team_name, ps.total_goals, ps.total_victories
FROM player_stats_with_id ps
LEFT JOIN teams t ON ps.player_id = t.team_id;
'''
display(pd.read_sql(join_view_query, connection))

# Conclusion 📖

## Conclusion

- Today we learned about different types of joins in SQL (`INNER`, `LEFT`, `RIGHT` (emulated), `FULL OUTER` (emulated)), noting potential version requirements in SQLite for direct support.
- We also learned about special joins: `CROSS JOIN` for combinations and `SELF JOIN` for relating rows within the same table.
- We saw how to merge tables vertically (by row) with `UNION`, `UNION ALL`, `INTERSECT`, and `EXCEPT`.
- We also saw the SQLite alternative to `MERGE`, the `INSERT ... ON CONFLICT` (UPSERT) clause 🚀.
- We saw how to create and use views in SQLite, which are virtual tables based on `SELECT` queries.

# And that's all for today! 🎉

# Thank you and have a great rest of your day! 🙏

In [ ]:
# Clean up: Close the connection
if 'connection' in locals() and connection:
    connection.close()
    print("\nSQLite connection closed.")